# 資料前處理：
## I.  Resize image as (224, 224)
## II. Rotate the image by angle (degree = 15)
## III. Horizontally flip the image with given probability (p=0.5)
## IV. Normalize the image with mean [0.485, 0.456, 0.406] and standard deviation [0.229, 0.224, 0.225]

In [2]:
import os
import numpy as np
import re
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import gc


In [3]:
import PIL
import torch
import torchvision

In [4]:
from torch.utils.data import Dataset, DataLoader
import cv2
from PIL import Image

class Dataset(Dataset):

    def __init__(self, x, y, transform):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        print(self.x[idx])
        img = cv2.imread(self.x[idx], cv2.IMREAD_COLOR)
        img = Image.fromarray(img)
        img = self.transform(img)
        return img, self.y[idx]
    
    

In [5]:
def load_mango_csv(csv_path='./C2_TrainDev/train.csv'):
    label2idx = {
    '不良-乳汁吸附': 0,
    '不良-機械傷害': 1,
    '不良-炭疽病': 2,
    '不良-著色不佳': 3,
    '不良-黑斑病': 4
    }
    path = []
    box = []
    label = []
    subdir = csv_path.split('/')[-1].split('.')[0].capitalize() #[-1]意思是倒數最後一col，.capitalize()將首英文字母大寫，其他小寫
    # subdir : Train , Dev
    # 此時我需要的輸出格式為：
    # path: 照片路徑 : ./C2_TrainDev/Train/img.jpg
    # label: 標籤  : [1,0,0,0,0]
    with open(csv_path, 'r', encoding='utf8') as f:        
        for line in f:
            clean_line = re.sub(',+\n', '', line).replace('\n', '').replace('\ufeff', '').split(',')
            curr_img_path = f'./C2_TrainDev/{subdir}/{clean_line[0]}'
            column = 5
            curr_label = [0,0,0,0,0]
            while column <= len(clean_line):
              symptom = clean_line[column]
              if symptom in label2idx:
                curr_label[label2idx[symptom]] = 1
              column += 5
            path.append(curr_img_path)
            label.append(curr_label)
    print('data size: ')
    print(len(path), len(label))
    print(path[:6])
    print(label[:6])
    count = np.zeros(5)
    for check in label:
      count += np.array(check)
    print('不良-乳汁吸附：'+str(count[0])+' '+str(count[0]/len(label)))
    print('不良-機械傷害：'+str(count[1])+' '+str(count[1]/len(label)))
    print('不良-炭疽病：'+str(count[2])+' '+str(count[2]/len(label)))
    print('不良-著色不佳：'+str(count[3])+' '+str(count[3]/len(label)))
    print('不良-黑斑病：'+str(count[4])+' '+str(count[4]/len(label)))
    print()
    return path, label

In [9]:
def dataloader_prepare(csv_path):
"""aug:
    csv_path:讀入資料之位置
"""
    transform_flip = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.RandomHorizontalFlip(p = 1),
        torchvision.transforms.RandomRotation(15, resample=PIL.Image.BILINEAR),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

    transform_rotation = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.RandomRotation((10,15), resample=PIL.Image.BILINEAR),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])



    path, label = load_mango_csv(csv_path)

    dataloader_flip = DataLoader( Dataset(path, label, transform_flip) , batch_size=1, shuffle=True)
    print(len(dataloader_flip))

    dataloader_origin = DataLoader( Dataset(path, label, transform) , batch_size=1, shuffle=True)
    print(len(dataloader_origin))

    dataloader_rotation = DataLoader( Dataset(path, label, transform_rotation) , batch_size=1, shuffle=True)
    print(len(dataloader_rotation))
    
    return dataloader_origin, dataloader_flip, dataloader_rotation


data size: 
25768 25768
['./C2_TrainDev/Train/38414.jpg', './C2_TrainDev/Train/03182.jpg', './C2_TrainDev/Train/29863.jpg', './C2_TrainDev/Train/17937.jpg', './C2_TrainDev/Train/40878.jpg', './C2_TrainDev/Train/29064.jpg']
[[0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 1, 0, 0]]
不良-乳汁吸附：2122.0 0.08235020180068302
不良-機械傷害：419.0 0.01626047811238746
不良-炭疽病：11489.0 0.44586308599813723
不良-著色不佳：14515.0 0.5632955603849736
不良-黑斑病：953.0 0.036983855945358586

25768
25768
25768


# The Image Feature Extractors


In [37]:
import torchvision.models as models
import torch.nn as nn
from ipywidgets import IntProgress

def feature_extractor(dataloader):
    print(len(dataloader))
    x = []
    y = []
    
    alexnet = models.alexnet(pretrained=True)
    vgg16 = models.vgg16(pretrained=True)
    
    for data in iter(dataloader):
        with torch.no_grad():
            alex_output = alexnet(data[0]).tolist()

        with torch.no_grad():
            vgg_output = vgg16(data[0]).tolist()
        
        x.append(alex_output[0]+vgg_output[0])
        
        new_data = []
        for data_ in data[1]:
            data_ =  int(data_)
            new_data.append(data_)
        y.append(new_data)
        
    print(np.array(x).shape)
    print(np.array(y).shape)
    return x, y

In [38]:
def prepare_new_data(csv_path):
"""aug:
    csv_path:讀入資料之位置
"""
    dataloader_origin, dataloader_flip, dataloader_rotation = dataloader_prepare(csv_path)
    
    x_origin, y_origin = feature_extractor(dataloader_origin)
    x_flip, y_flip = feature_extractor(dataloader_flip)
    x_rotation, y_rotation = feature_extractor(dataloader_rotation)
    x = x_origin + x_flip + x_rotation
    y = y_origin + y_flip + y_rotation
    
    return x, y 
    

25768
./C2_TrainDev/Train/39585.jpg
./C2_TrainDev/Train/06157.jpg
./C2_TrainDev/Train/35379.jpg
./C2_TrainDev/Train/04741.jpg
./C2_TrainDev/Train/15135.jpg
./C2_TrainDev/Train/26838.jpg
./C2_TrainDev/Train/36793.jpg
./C2_TrainDev/Train/17629.jpg
./C2_TrainDev/Train/16693.jpg
./C2_TrainDev/Train/14097.jpg
./C2_TrainDev/Train/35482.jpg
./C2_TrainDev/Train/26534.jpg
./C2_TrainDev/Train/05051.jpg
./C2_TrainDev/Train/12951.jpg
./C2_TrainDev/Train/16295.jpg
./C2_TrainDev/Train/45538.jpg
./C2_TrainDev/Train/20010.jpg
./C2_TrainDev/Train/38075.jpg
./C2_TrainDev/Train/26075.jpg
./C2_TrainDev/Train/49246.jpg
./C2_TrainDev/Train/32611.jpg
./C2_TrainDev/Train/07927.jpg
./C2_TrainDev/Train/05786.jpg
./C2_TrainDev/Train/37956.jpg
./C2_TrainDev/Train/43174.jpg
./C2_TrainDev/Train/12768.jpg
./C2_TrainDev/Train/07090.jpg
./C2_TrainDev/Train/20701.jpg
./C2_TrainDev/Train/27742.jpg
./C2_TrainDev/Train/04571.jpg
./C2_TrainDev/Train/22992.jpg
./C2_TrainDev/Train/35399.jpg
./C2_TrainDev/Train/06475.jpg
./C2

./C2_TrainDev/Train/36144.jpg
./C2_TrainDev/Train/08183.jpg
./C2_TrainDev/Train/09370.jpg
./C2_TrainDev/Train/03409.jpg
./C2_TrainDev/Train/41498.jpg
./C2_TrainDev/Train/08603.jpg
./C2_TrainDev/Train/10300.jpg
./C2_TrainDev/Train/08959.jpg
./C2_TrainDev/Train/00619.jpg
./C2_TrainDev/Train/43784.jpg
./C2_TrainDev/Train/41054.jpg
./C2_TrainDev/Train/11662.jpg
./C2_TrainDev/Train/13563.jpg
./C2_TrainDev/Train/05776.jpg
./C2_TrainDev/Train/24250.jpg
./C2_TrainDev/Train/48150.jpg
./C2_TrainDev/Train/16280.jpg
./C2_TrainDev/Train/28764.jpg
./C2_TrainDev/Train/39296.jpg
./C2_TrainDev/Train/16546.jpg
./C2_TrainDev/Train/07336.jpg
./C2_TrainDev/Train/15128.jpg
./C2_TrainDev/Train/18200.jpg
./C2_TrainDev/Train/14743.jpg
./C2_TrainDev/Train/11823.jpg
./C2_TrainDev/Train/45719.jpg
./C2_TrainDev/Train/12605.jpg
./C2_TrainDev/Train/38887.jpg
./C2_TrainDev/Train/24237.jpg
./C2_TrainDev/Train/32356.jpg
./C2_TrainDev/Train/02259.jpg
./C2_TrainDev/Train/21726.jpg
./C2_TrainDev/Train/13370.jpg
./C2_Train

./C2_TrainDev/Train/03747.jpg
./C2_TrainDev/Train/12676.jpg
./C2_TrainDev/Train/35702.jpg
./C2_TrainDev/Train/05408.jpg
./C2_TrainDev/Train/22279.jpg
./C2_TrainDev/Train/17800.jpg
./C2_TrainDev/Train/22968.jpg
./C2_TrainDev/Train/06899.jpg
./C2_TrainDev/Train/14033.jpg
./C2_TrainDev/Train/34973.jpg
./C2_TrainDev/Train/07362.jpg
./C2_TrainDev/Train/24522.jpg
./C2_TrainDev/Train/18639.jpg
./C2_TrainDev/Train/30259.jpg
./C2_TrainDev/Train/09124.jpg
./C2_TrainDev/Train/48952.jpg
./C2_TrainDev/Train/05821.jpg
./C2_TrainDev/Train/47452.jpg
./C2_TrainDev/Train/19181.jpg
./C2_TrainDev/Train/08160.jpg
./C2_TrainDev/Train/18229.jpg
./C2_TrainDev/Train/03476.jpg
./C2_TrainDev/Train/40082.jpg
./C2_TrainDev/Train/21943.jpg
./C2_TrainDev/Train/41917.jpg
./C2_TrainDev/Train/04127.jpg
./C2_TrainDev/Train/27633.jpg
./C2_TrainDev/Train/31407.jpg
./C2_TrainDev/Train/20786.jpg
./C2_TrainDev/Train/25972.jpg
./C2_TrainDev/Train/40809.jpg
./C2_TrainDev/Train/47490.jpg
./C2_TrainDev/Train/44836.jpg
./C2_Train

./C2_TrainDev/Train/02369.jpg
./C2_TrainDev/Train/33273.jpg
./C2_TrainDev/Train/40769.jpg
./C2_TrainDev/Train/05907.jpg
./C2_TrainDev/Train/06240.jpg
./C2_TrainDev/Train/07428.jpg
./C2_TrainDev/Train/05968.jpg
./C2_TrainDev/Train/02713.jpg
./C2_TrainDev/Train/38882.jpg
./C2_TrainDev/Train/19413.jpg
./C2_TrainDev/Train/22028.jpg
./C2_TrainDev/Train/40320.jpg
./C2_TrainDev/Train/10573.jpg
./C2_TrainDev/Train/08764.jpg
./C2_TrainDev/Train/35082.jpg
./C2_TrainDev/Train/47469.jpg
./C2_TrainDev/Train/36959.jpg
./C2_TrainDev/Train/28317.jpg
./C2_TrainDev/Train/29164.jpg
./C2_TrainDev/Train/15318.jpg
./C2_TrainDev/Train/07489.jpg
./C2_TrainDev/Train/22349.jpg
./C2_TrainDev/Train/12798.jpg
./C2_TrainDev/Train/06627.jpg
./C2_TrainDev/Train/18777.jpg
./C2_TrainDev/Train/15856.jpg
./C2_TrainDev/Train/24928.jpg
./C2_TrainDev/Train/34494.jpg
./C2_TrainDev/Train/06737.jpg
./C2_TrainDev/Train/06112.jpg
./C2_TrainDev/Train/21488.jpg
./C2_TrainDev/Train/43955.jpg
./C2_TrainDev/Train/05029.jpg
./C2_Train

./C2_TrainDev/Train/33123.jpg
./C2_TrainDev/Train/49985.jpg
./C2_TrainDev/Train/14970.jpg
./C2_TrainDev/Train/36249.jpg
./C2_TrainDev/Train/22830.jpg
./C2_TrainDev/Train/00479.jpg
./C2_TrainDev/Train/35430.jpg
./C2_TrainDev/Train/45404.jpg
./C2_TrainDev/Train/23766.jpg
./C2_TrainDev/Train/21310.jpg
./C2_TrainDev/Train/41560.jpg
./C2_TrainDev/Train/13483.jpg
./C2_TrainDev/Train/19899.jpg
./C2_TrainDev/Train/22052.jpg
./C2_TrainDev/Train/03353.jpg
./C2_TrainDev/Train/12509.jpg
./C2_TrainDev/Train/05311.jpg
./C2_TrainDev/Train/00152.jpg
./C2_TrainDev/Train/30442.jpg
./C2_TrainDev/Train/19849.jpg
./C2_TrainDev/Train/02576.jpg
./C2_TrainDev/Train/02721.jpg
./C2_TrainDev/Train/35145.jpg
./C2_TrainDev/Train/44938.jpg
./C2_TrainDev/Train/29278.jpg
./C2_TrainDev/Train/03926.jpg
./C2_TrainDev/Train/27003.jpg
./C2_TrainDev/Train/02072.jpg
./C2_TrainDev/Train/08975.jpg
./C2_TrainDev/Train/00896.jpg
./C2_TrainDev/Train/27025.jpg
./C2_TrainDev/Train/12962.jpg
./C2_TrainDev/Train/19253.jpg
./C2_Train

...

./C2_TrainDev/Train/18722.jpg
./C2_TrainDev/Train/08814.jpg
./C2_TrainDev/Train/37922.jpg
./C2_TrainDev/Train/31156.jpg
./C2_TrainDev/Train/35903.jpg
./C2_TrainDev/Train/33434.jpg
./C2_TrainDev/Train/43106.jpg
./C2_TrainDev/Train/09618.jpg
./C2_TrainDev/Train/42246.jpg
./C2_TrainDev/Train/02665.jpg
./C2_TrainDev/Train/27667.jpg
./C2_TrainDev/Train/12581.jpg
./C2_TrainDev/Train/04487.jpg
./C2_TrainDev/Train/34526.jpg
./C2_TrainDev/Train/10933.jpg
./C2_TrainDev/Train/19312.jpg
./C2_TrainDev/Train/32011.jpg
./C2_TrainDev/Train/10870.jpg
./C2_TrainDev/Train/40405.jpg
./C2_TrainDev/Train/03735.jpg
./C2_TrainDev/Train/15691.jpg
./C2_TrainDev/Train/47881.jpg
./C2_TrainDev/Train/41524.jpg
./C2_TrainDev/Train/08371.jpg
./C2_TrainDev/Train/21623.jpg
./C2_TrainDev/Train/32950.jpg
./C2_TrainDev/Train/16870.jpg
./C2_TrainDev/Train/42733.jpg
./C2_TrainDev/Train/39615.jpg
./C2_TrainDev/Train/45700.jpg
./C2_TrainDev/Train/33828.jpg
./C2_TrainDev/Train/15889.jpg
./C2_TrainDev/Train/39109.jpg
./C2_Train

./C2_TrainDev/Train/44896.jpg
./C2_TrainDev/Train/37198.jpg
./C2_TrainDev/Train/45554.jpg
./C2_TrainDev/Train/29307.jpg
./C2_TrainDev/Train/40919.jpg
./C2_TrainDev/Train/25067.jpg
./C2_TrainDev/Train/22642.jpg
./C2_TrainDev/Train/00390.jpg
./C2_TrainDev/Train/03308.jpg
./C2_TrainDev/Train/22933.jpg
./C2_TrainDev/Train/32062.jpg
./C2_TrainDev/Train/29503.jpg
./C2_TrainDev/Train/43624.jpg
./C2_TrainDev/Train/08700.jpg
./C2_TrainDev/Train/25534.jpg
./C2_TrainDev/Train/17919.jpg
./C2_TrainDev/Train/47491.jpg
./C2_TrainDev/Train/28856.jpg
./C2_TrainDev/Train/32876.jpg
./C2_TrainDev/Train/06504.jpg
./C2_TrainDev/Train/39301.jpg
./C2_TrainDev/Train/00348.jpg
./C2_TrainDev/Train/21149.jpg
./C2_TrainDev/Train/15894.jpg
./C2_TrainDev/Train/25978.jpg
./C2_TrainDev/Train/44639.jpg
./C2_TrainDev/Train/07094.jpg
./C2_TrainDev/Train/17568.jpg
./C2_TrainDev/Train/46299.jpg
./C2_TrainDev/Train/23281.jpg
./C2_TrainDev/Train/06457.jpg
./C2_TrainDev/Train/35630.jpg
./C2_TrainDev/Train/33232.jpg
./C2_Train

./C2_TrainDev/Train/35332.jpg
./C2_TrainDev/Train/19125.jpg
./C2_TrainDev/Train/40745.jpg
./C2_TrainDev/Train/35252.jpg
./C2_TrainDev/Train/34260.jpg
./C2_TrainDev/Train/33912.jpg
./C2_TrainDev/Train/15916.jpg
./C2_TrainDev/Train/40237.jpg
./C2_TrainDev/Train/40754.jpg
./C2_TrainDev/Train/36639.jpg
./C2_TrainDev/Train/19929.jpg
./C2_TrainDev/Train/49814.jpg
./C2_TrainDev/Train/49218.jpg
./C2_TrainDev/Train/46733.jpg
./C2_TrainDev/Train/02061.jpg
./C2_TrainDev/Train/32334.jpg
./C2_TrainDev/Train/35106.jpg
./C2_TrainDev/Train/11102.jpg
./C2_TrainDev/Train/29620.jpg
./C2_TrainDev/Train/05962.jpg
./C2_TrainDev/Train/21153.jpg
./C2_TrainDev/Train/06944.jpg
./C2_TrainDev/Train/17617.jpg
./C2_TrainDev/Train/18720.jpg
./C2_TrainDev/Train/01815.jpg
./C2_TrainDev/Train/27527.jpg
./C2_TrainDev/Train/03419.jpg
./C2_TrainDev/Train/48773.jpg
./C2_TrainDev/Train/37438.jpg
./C2_TrainDev/Train/36330.jpg
./C2_TrainDev/Train/16103.jpg
./C2_TrainDev/Train/05886.jpg
./C2_TrainDev/Train/05625.jpg
./C2_Train

./C2_TrainDev/Train/43192.jpg
./C2_TrainDev/Train/21448.jpg
./C2_TrainDev/Train/43051.jpg
./C2_TrainDev/Train/48793.jpg
./C2_TrainDev/Train/11813.jpg
./C2_TrainDev/Train/34507.jpg
./C2_TrainDev/Train/48136.jpg
./C2_TrainDev/Train/12860.jpg
./C2_TrainDev/Train/13051.jpg
./C2_TrainDev/Train/28123.jpg
./C2_TrainDev/Train/22289.jpg
./C2_TrainDev/Train/33311.jpg
./C2_TrainDev/Train/33865.jpg
./C2_TrainDev/Train/05092.jpg
./C2_TrainDev/Train/45148.jpg
./C2_TrainDev/Train/00965.jpg
./C2_TrainDev/Train/28506.jpg
./C2_TrainDev/Train/28559.jpg
./C2_TrainDev/Train/01971.jpg
./C2_TrainDev/Train/45769.jpg
./C2_TrainDev/Train/00947.jpg
./C2_TrainDev/Train/11324.jpg
./C2_TrainDev/Train/36685.jpg
./C2_TrainDev/Train/05194.jpg
./C2_TrainDev/Train/39507.jpg
./C2_TrainDev/Train/45029.jpg
./C2_TrainDev/Train/45742.jpg
./C2_TrainDev/Train/49385.jpg
./C2_TrainDev/Train/25565.jpg
./C2_TrainDev/Train/23573.jpg
./C2_TrainDev/Train/11295.jpg
./C2_TrainDev/Train/43581.jpg
./C2_TrainDev/Train/48796.jpg
./C2_Train

./C2_TrainDev/Train/31455.jpg
./C2_TrainDev/Train/41703.jpg
./C2_TrainDev/Train/35591.jpg
./C2_TrainDev/Train/32552.jpg
./C2_TrainDev/Train/15449.jpg
./C2_TrainDev/Train/15363.jpg
./C2_TrainDev/Train/26315.jpg
./C2_TrainDev/Train/09609.jpg
./C2_TrainDev/Train/33946.jpg
./C2_TrainDev/Train/32247.jpg
./C2_TrainDev/Train/06207.jpg
./C2_TrainDev/Train/36389.jpg
./C2_TrainDev/Train/49455.jpg
./C2_TrainDev/Train/06909.jpg
./C2_TrainDev/Train/36174.jpg
./C2_TrainDev/Train/46687.jpg
./C2_TrainDev/Train/18134.jpg
./C2_TrainDev/Train/16776.jpg
./C2_TrainDev/Train/13272.jpg
./C2_TrainDev/Train/21276.jpg
./C2_TrainDev/Train/43877.jpg
./C2_TrainDev/Train/34197.jpg
./C2_TrainDev/Train/20983.jpg
./C2_TrainDev/Train/06725.jpg
./C2_TrainDev/Train/05936.jpg
./C2_TrainDev/Train/23226.jpg
./C2_TrainDev/Train/41891.jpg
./C2_TrainDev/Train/08821.jpg
./C2_TrainDev/Train/07501.jpg
./C2_TrainDev/Train/41296.jpg
./C2_TrainDev/Train/13009.jpg
./C2_TrainDev/Train/00670.jpg
./C2_TrainDev/Train/34061.jpg
./C2_Train

In [2]:
import json
def write_data(file_name):
    x_train, y_train = prepare_new_data(csv_path='./C2_TrainDev/train.csv')
    x_dev, y_dev = prepare_new_data(csv_path='./C2_TrainDev/dev.csv')

    with open(f'{file_name}/x_train.txt', 'w') as outfile:
        json.dump(x_train, outfile)
    with open(f'{file_name}/y_train.txt', 'w') as outfile:
        json.dump(y_train, outfile)
    with open(f'{file_name}/x_dev.txt', 'w') as outfile:
        json.dump(x_dev, outfile)
    with open(f'{file_name}/y_dev.txt', 'w') as outfile:
        json.dump(y_dev, outfile)
    return x_train, y_train, x_dev, y_dev
        
def read_data(file_name):
    with open(f'{file_name}/x_train.txt') as jsonfile:
        x_train = json.load(jsonfile)
    with open(f'{file_name}/y_train.txt') as jsonfile:
        y_train = json.load(jsonfile)
    with open(f'{file_name}/x_dev.txt') as jsonfile:
        x_dev = json.load(jsonfile)
    with open(f'{file_name}/y_dev.txt') as jsonfile:
        y_dev = json.load(jsonfile)
    return x_train, y_train, x_dev, y_dev

In [3]:
def prepare_data(file_name):
    if not os.path.isdir(file_name):
        os.makedirs(file_name)
        x_train, y_train, x_dev, y_dev = write_data(file_name)
    else:
        x_train, y_train, x_dev, y_dev = read_data(file_name)
    
    return x_train, y_train, x_dev, y_dev

In [ ]:
file_name = 'baseline_vgg16feature_anova_svm'
x_train, y_train, x_dev, y_dev = prepare_data(file_name)

# Defective Classification
## anova, svm

In [52]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import matthews_corrcoef, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
    
    
# ANOVA SVM-C
# 1) anova filter, take 500 best ranked features
anova_filter = SelectKBest(f_regression, k=500)
# 2) svm
clf = LinearSVC(random_state=42, C=1.0, class_weight='balanced')
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(x_train, y_train)
y_dev_pred = anova_svm.predict(x_dev)

print('evaluating dev data...')
cm = confusion_matrix(y_dev, y_dev_pred)
acc = accuracy_score(y_dev, y_dev_pred)
f1 = f1_score(y_dev, y_dev_pred)
p = precision_score(y_dev, y_dev_pred)
r = recall_score(y_dev, y_dev_pred)

print(cm, acc, f1, p, r)

ValueError: bad input shape (77304, 5)